### Importaciones de paquetes necesarios

In [134]:
from deap import creator, base, tools, algorithms

import random
import numpy as np
import pandas as pd


### Creación del turno de enfermería

In [32]:
from turnosenfermeria import TurnosEnfermeria

In [33]:
preferencias = pd.read_csv("./data/datos_enfermeria.txt", sep=' ')
print (preferencias)

   Nombre  Pref_M  Pref_T  Pref_N
0   Enf_A       1       1       0
1   Enf_B       1       0       1
2   Enf_C       1       1       1
3   Enf_D       0       1       0
4   Enf_E       0       0       1
5   Enf_F       0       1       1
6   Enf_G       1       1       0
7   Enf_H       0       0       1
8   Enf_I       1       1       1
9   Enf_J       1       1       0
10  Enf_K       1       1       1
11  Enf_L       1       1       0
12  Enf_M       1       0       1


In [34]:
enfermeras = preferencias["Nombre"]
print (enfermeras)

0     Enf_A
1     Enf_B
2     Enf_C
3     Enf_D
4     Enf_E
5     Enf_F
6     Enf_G
7     Enf_H
8     Enf_I
9     Enf_J
10    Enf_K
11    Enf_L
12    Enf_M
Name: Nombre, dtype: object


In [60]:
turnos = TurnosEnfermeria (enfermeras, preferencias)




273

# Algoritmo Genético Simple

In [258]:
tamano_individuo = 21
tamano_calendario = len (preferencias)
tamano_poblacion = 50
max_generaciones = 100
p_cruce = 0.9
p_mutacion = 0.1

toolbox = base.Toolbox()

## Vamos a crear el Algoritmo Genético Simple a ver que nos devuelve

In [266]:
def CreaccionCalendarioTurnos (tamano_individuo, tamano_poblacion, p_cruce, p_mutacion, max_generaciones, verbose=True):
    toolbox=base.Toolbox()
    creator.create("ClaseAjusteMin", base.Fitness, weights=(-1.0,))
    creator.create("ClaseIndividuo", list, fitness=creator.ClaseAjusteMin)

    toolbox.register("ceroOrUno", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.ClaseIndividuo, toolbox.ceroOrUno, tamano_individuo*tamano_calendario)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual, tamano_poblacion)
    poblacion = toolbox.population()
   


    toolbox.register("evaluate", turnos.getCoste)


    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/tamano_individuo)


    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("max", np.max)
    stats.register("min", np.min)


    hof = tools.HallOfFame(1)
    poblacion_final, logbook= algorithms.eaSimple(poblacion, toolbox,
                                                 cxpb=p_cruce, mutpb=p_mutacion,
                                                 ngen=max_generaciones,
                                                 stats=stats,halloffame=hof,
                                                 verbose=verbose)

    
    turnos.mostrarInfoCalendario (hof[0])
    
    return poblacion_final, logbook

In [267]:
# Comprobación del funcionamiento del algoritmo


poblacion_final, logbook = CreaccionCalendarioTurnos (tamano_individuo, tamano_poblacion, p_cruce, p_mutacion, max_generaciones)

gen	nevals	avg    	std    	max   	min   
0  	50    	235.827	32.3276	312.48	166.88
1  	49    	211.187	24.449 	258.72	165.76
2  	50    	191.677	20.791 	243.04	160.16
3  	46    	173.981	14.8848	204.96	145.6 
4  	45    	160.339	10.3195	187.04	140   
5  	46    	152.611	10.1391	187.04	136.64
6  	49    	147.28 	9.30409	171.36	127.68
7  	44    	140.806	12.2834	173.6 	117.6 
8  	46    	129.114	10.882 	151.2 	104.16
9  	44    	121.005	8.94588	140   	103.04
10 	46    	114.733	9.78744	138.88	95.2  
11 	48    	105.974	7.63866	128.8 	94.08 
12 	48    	102.906	7.9026 	126.56	85.12 
13 	42    	99.6576	7.38687	131.04	85.12 
14 	41    	95.7152	6.53147	117.6 	82.88 
15 	40    	91.9296	6.74921	124.32	78.4  
16 	46    	88.6368	5.54602	104.16	78.4  
17 	46    	84.2688	4.52214	99.68 	77.28 
18 	47    	83.888 	9.8056 	123.2 	69.44 
19 	48    	79.4304	7.88416	108.64	66.08 
20 	45    	76.048 	7.13731	103.04	60.48 
21 	46    	70.8512	5.42159	92.96 	60.48 
22 	46    	70     	8.93547	113.12	60.48 
23 	43    	66.59

## Mostramos el calendario por pantalla